Raw data, (>29,000 bp), 28.03.2020 22:39

2367 genomes

gisaid_cov2020_sequences_complete.fasta

In [13]:
!pwd

/home/mike/Projects_Bioinf/Viruses/covid19


In [1]:
#Remove spaces and some characters from contig names:
!sed -i 's/ //g' gisaid_cov2020_sequences_complete.fasta
!sed -i 's/[|]/_/g' gisaid_cov2020_sequences_complete.fasta
!sed -i 's/[/]/_/g' gisaid_cov2020_sequences_complete.fasta

sed: can't read gisaid_cov2020_sequences_complete.fasta: No such file or directory
sed: can't read gisaid_cov2020_sequences_complete.fasta: No such file or directory
sed: can't read gisaid_cov2020_sequences_complete.fasta: No such file or directory


In [ ]:
#Split to separate genomes with Seqretsplit from EMBOSS
!mkdir splitted
!cd splitted
!seqretsplit  ../gisaid_cov2020_sequences_complete.fasta 

In [ ]:
#Create vcfs 
!mkdir vcfs
!for i in $(ls splitted/*.fasta); do minimap2 -c --cs nCoV-2019.reference.fasta  //
!$i | sort -k6,6 -k8,8n | /Nancy/mrayko/Libs/minimap2/misc/paftools.js call -L20000 //
!-f nCoV-2019.reference.fasta  - > vcfs/$(basename $i).vcf ;done

In [2]:
#Merge to single vcf
!ls -d vcfs/* > vcfs.list 
!sed -i '/_pangolin_/d' vcfs.list 
!sed -i '/_bat_/d' vcfs.list 

!java -jar picard.jar MergeVcfs I=vcfs.list O=merged.vcf

In [ ]:
#Select only SNV that remained singletons since March 1.
!python3 /home/dantipov/scripts/covid/covid19_variation_analysis/find_singleton_mutations.py  merged.vcf 2020-03-01 > singletons_before_2020-03-01.vcf

In [15]:
# Keep only non-singleton SNVs.
infile = "merged.vcf"
table = {}
with open (infile, "r") as input_file:
    lines = [i.strip() for i in input_file.readlines()]

for line in lines:
        arr = line.split()
        if arr[0] == "NC_045512.2":
            id = arr[1] + "_" + arr[3] + "_" + arr[4]
            if  not (id in table.keys()):
                table[id] = []
            table[id].append(line.strip())

# Write non-singleton snps
with open("merged_non_singletons.vcf", "w") as output:
  for line in lines:
    arr = line.split()
    if arr[0] == "NC_045512.2" and len(table[arr[1] + "_" + arr[3] + "_" + arr[4]])==1:
        continue
    else:
        #output.append(line)
        output.write("%s\n" % line)

# Write uniq non_singleton snps       
seen = set()
with open("merged_non_singletons_uniq.vcf", "w") as output:
  for line in lines:
    arr = line.split()
    if arr[0] != "NC_045512.2":
        output.write("%s\n" % line)
    else:
        id = arr[1] + "_" + arr[3] + "_" + arr[4]
        if len(table[id])==1:
            continue
        else:
            if id not in seen:
                seen.add(id)
                arr[7]=arr[7]+";"+str(len(table[id]))
                #line = line + "\t" +str(len(table[id]))
                output.write("%s\n" % '\t'.join(arr))



In [1]:
# Remove dot from header for SnpEff
!sed -i 's/NC_045512.2/NC_045512/g'  merged_non_singletons_uniq.vcf
# Run SnpEff
! java -jar snpEff.jar covid merged_non_singletons_uniq.vcf  > merged_non_singletons_uniq_annotated.vcf 

/home/mike/Projects_Bioinf/Viruses/covid19/repeat_1_april


__conservative_inframe_deletion	 	4	0.115%__  
__disruptive_inframe_deletion	 	6	0.172%__  
__disruptive_inframe_insertion	 	1	0.029%__  
downstream_gene_variant	 	1,440	41.308%  
__frameshift_variant	 	2	0.057%__  
gene_fusion	 	1	0.029%  
intergenic_region	 	82	2.352%  
missense_variant	 	395	11.331%  
__start_lost	 	1	0.029%__  
__stop_gained	 	5	0.143%__  
synonymous_variant	 	249	7.143%  
upstream_gene_variant	 	1,300	37.292%  
